In [1]:
from simnet import *

In [2]:
def temp2(mode=0):
    G = nx.Graph()
    pylab.figure(figsize=(10, 10))
    scoredic = dict()
    f = MCS_Finder("Streptomyces")
    Clist = f.get_Cnlist_from_label2("C(C)-C(C)-C(C-C-C-C-C-C)-C-C-C-C-C-C-C-C-C")
    for Cnumber in Clist:
        filepath = "SIMCOMP/" + Cnumber + ".txt"
        if not os.path.exists(filepath):
            continue
        with open(filepath, "r") as fi:
            scorelist = []
            score = fi.readline().split("\t")
            score[1] = float(score[1][:-1])
            if mode == 1:
                while(score[1] > 0.9):
                    scorelist.append(score)
                    if score[0] in Clist and score[1] < 1:
                        G = nxappend(G, Cnumber, score[0], score[1])
                    score = fi.readline().split("\t")
                    score[1] = float(score[1][:-1])
            elif mode == 2:
                try:
                    while(True):
                        scorelist.append(score)
                        if score[0] in Clist and score[1] < 1:
                            G = nxappend(G, Cnumber, score[0], score[1])
                        score = fi.readline().split("\t")
                        score[1] = float(score[1][:-1])
                except IndexError:
                    pass
            else:
                for i in range(3):
                    scorelist.append(score)
                    if score[0] in Clist and score[1] < 1:
                        G = nxappend(G, Cnumber, score[0], score[1])
                    score = fi.readline().split("\t")
                    score[1] = float(score[1][:-1])
            scoredic[Cnumber] = scorelist
    print(str(len(G.nodes())) + "/" + str(len(Clist)))
    print("edge:" + str(len(G.edges())))
    return G
    for g in nx.connected_component_subgraphs(G):
        nx.draw_networkx(g)
        plt.xticks([])
        plt.yticks([])
        plt.show()

In [3]:
G = temp2()

20/20
edge:23


In [4]:
Glist = list(nx.connected_component_subgraphs(G))

In [5]:
dic = dict()
for g in Glist:
    dic[g] = len(g.nodes())

In [6]:
sortGlist = sorted(dic.items(), key=lambda x: x[1], reverse=True)
print(sortGlist)

[(<networkx.classes.graph.Graph object at 0x1138092e8>, 13), (<networkx.classes.graph.Graph object at 0x1138099b0>, 5), (<networkx.classes.graph.Graph object at 0x113809f98>, 2)]


In [7]:
sortGlist[1][0].nodes()

['C00017726', 'C00026595', 'C00026596', 'C00015228', 'C00015229']

In [2]:
f = MCS_Finder("Streptomyces")
Clist = f.get_Cnlist_from_label2("C(C)-C(C)-C(C-C-C-C-C-C)-C-C-C-C-C-C-C-C-C")

In [3]:
print(Clist)

['C00015228', 'C00015229', 'C00016476', 'C00016751', 'C00016938', 'C00017103', 'C00017237', 'C00017544', 'C00017573', 'C00017671', 'C00017726', 'C00017791', 'C00017792', 'C00018127', 'C00024753', 'C00026595', 'C00026596', 'C00028349', 'C00028350', 'C00028707']


In [11]:
%%time
i = 0
for Cnumber in Clist:
    filepath = "SIMCOMP2/" + Cnumber + ".txt"
    if not os.path.exists(filepath):
        url = "http://rest.genome.jp/simcomp/"+ Cnumber + "/knapsack"
        i += 1
        print(url, i)

http://rest.genome.jp/simcomp/C00015228/knapsack 1
http://rest.genome.jp/simcomp/C00015229/knapsack 2
http://rest.genome.jp/simcomp/C00016476/knapsack 3
http://rest.genome.jp/simcomp/C00016751/knapsack 4
http://rest.genome.jp/simcomp/C00016938/knapsack 5
http://rest.genome.jp/simcomp/C00017103/knapsack 6
http://rest.genome.jp/simcomp/C00017237/knapsack 7
http://rest.genome.jp/simcomp/C00017544/knapsack 8
http://rest.genome.jp/simcomp/C00017573/knapsack 9
http://rest.genome.jp/simcomp/C00017671/knapsack 10
http://rest.genome.jp/simcomp/C00017726/knapsack 11
http://rest.genome.jp/simcomp/C00017791/knapsack 12
http://rest.genome.jp/simcomp/C00017792/knapsack 13
http://rest.genome.jp/simcomp/C00018127/knapsack 14
http://rest.genome.jp/simcomp/C00024753/knapsack 15
http://rest.genome.jp/simcomp/C00026595/knapsack 16
http://rest.genome.jp/simcomp/C00026596/knapsack 17
http://rest.genome.jp/simcomp/C00028349/knapsack 18
http://rest.genome.jp/simcomp/C00028350/knapsack 19
http://rest.genome.jp

In [16]:
import time
import urllib

In [28]:
%%time
i = 0
t = time.time()
dic = dict()
for Cnumber in Clist:
    filepath = "SIMCOMP2/" + Cnumber + ".txt"
    if not os.path.exists(filepath):
        amalist = Clist.copy()
        amalist.remove(Cnumber)
        url = "http://rest.genome.jp/simcomp2/"+ Cnumber + "/"
        for amali in amalist:
            url += amali + "+"
        url = url[:-1]
        url += "/cutoff=0.1"
        urllib.request.urlretrieve(url, filepath)
        i += 1
        print(url, i)
        print(time.time() - t)
        print(time.strftime("%X"))
        time.sleep(60)
        if i == 4:
            break

CPU times: user 527 µs, sys: 659 µs, total: 1.19 ms
Wall time: 644 µs


In [26]:
%%time
time.sleep(60)

CPU times: user 2.07 ms, sys: 2.18 ms, total: 4.25 ms
Wall time: 1min


SIMCOMP2を使って類似度を新しく出した。  
これなら約2秒なので30倍くらいの短縮になる。

In [33]:
def temp3(mode=0):
    G = nx.Graph()
    pylab.figure(figsize=(10, 10))
    scoredic = dict()
    f = MCS_Finder("Streptomyces")
    Clist = f.get_Cnlist_from_label2("C(C)-C(C)-C(C-C-C-C-C-C)-C-C-C-C-C-C-C-C-C")
    for Cnumber in Clist:
        filepath = "SIMCOMP2/" + Cnumber + ".txt"
        if not os.path.exists(filepath):
            continue
        with open(filepath, "r") as fi:
            scorelist = []
            score = fi.readline().split("\t")
            score[2] = float(score[2][:-1])
            if mode == 1:
                while(score[1] > 0.9):
                    scorelist.append(score)
                    if score[0] in Clist and score[2] < 1:
                        G = nxappend(G, Cnumber, score[0], score[1])
                    score = fi.readline().split("\t")
                    score[1] = float(score[1][:-1])
            elif mode == 2:
                try:
                    while(True):
                        scorelist.append(score)
                        if score[2] < 1:
                            G = nxappend(G, Cnumber, score[1], score[2])
                        score = fi.readline().split("\t")
                        score[2] = float(score[2][:-1])
                except IndexError:
                    pass
            else:
                for i in range(3):
                    scorelist.append(score)
                    if score[0] in Clist and score[1] < 1:
                        G = nxappend(G, Cnumber, score[0], score[1])
                    score = fi.readline().split("\t")
                    score[1] = float(score[1][:-1])
            scoredic[Cnumber] = scorelist
    print(str(len(G.nodes())) + "/" + str(len(Clist)))
    print("edge:" + str(len(G.edges())))
    return G
    for g in nx.connected_component_subgraphs(G):
        nx.draw_networkx(g)
        plt.xticks([])
        plt.yticks([])
        plt.show()

In [35]:
G1 = temp2(2)
G2 = temp3(2)

20/20
edge:190
20/20
edge:190


In [37]:
print(G1.nodes(data=True))
print(G1.edges(data=True))

[('C00017671', {}), ('C00026595', {}), ('C00017726', {}), ('C00017792', {}), ('C00016476', {}), ('C00017103', {}), ('C00016938', {}), ('C00015229', {}), ('C00017544', {}), ('C00028350', {}), ('C00016751', {}), ('C00028707', {}), ('C00017237', {}), ('C00018127', {}), ('C00024753', {}), ('C00015228', {}), ('C00028349', {}), ('C00026596', {}), ('C00017791', {}), ('C00017573', {})]
[('C00017671', 'C00017791', {'weight': 0.662818}), ('C00017671', 'C00028350', {'weight': 0.869369}), ('C00017671', 'C00026596', {'weight': 0.76}), ('C00017671', 'C00017726', {'weight': 0.785714}), ('C00017671', 'C00017792', {'weight': 0.676768}), ('C00017671', 'C00016476', {'weight': 0.944444}), ('C00017671', 'C00017103', {'weight': 0.637555}), ('C00017671', 'C00016938', {'weight': 0.770833}), ('C00017671', 'C00015229', {'weight': 0.772908}), ('C00017671', 'C00026595', {'weight': 0.779141}), ('C00017671', 'C00016751', {'weight': 0.900452}), ('C00017671', 'C00028707', {'weight': 0.945701}), ('C00017671', 'C000172

In [40]:
for n1, n2 in zip(G1.nodes(data=True), G2.nodes(data=True)):
    if n1 == n2:
        print("OK")
    else:
        print("dame")
        print(n1, n2)

OK
OK
OK
OK
OK
dame
('C00017103', {}) ('C00017791', {})
OK
OK
dame
('C00017544', {}) ('C00028350', {})
dame
('C00028350', {}) ('C00016751', {})
dame
('C00016751', {}) ('C00017103', {})
OK
OK
OK
OK
OK
OK
OK
dame
('C00017791', {}) ('C00017544', {})
OK


In [42]:
l1 = list()
l2 = list()
for e1, e2 in zip(G1.edges(data=True), G2.edges(data=True)):
    if e1 == e2:
        print("OK")
    else:
        print("dame")
        print(e1, e2)
        l1.append(e1)
        l2.append(e2)

OK
OK
OK
dame
('C00017671', 'C00017726', {'weight': 0.785714}) ('C00017671', 'C00015228', {'weight': 0.792261})
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
dame
('C00017671', 'C00015228', {'weight': 0.763527}) ('C00017671', 'C00017726', {'weight': 0.785714})
OK
OK
OK
OK
OK
dame
('C00026595', 'C00015228', {'weight': 0.973684}) ('C00026595', 'C00015228', {'weight': 0.973392})
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
dame
('C00017726', 'C00015228', {'weight': 0.921053}) ('C00017726', 'C00015228', {'weight': 0.912088})
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
dame
('C00017792', 'C00028350', {'weight': 0.805869}) ('C00017792', 'C00017791', {'weight': 0.720698})
dame
('C00017792', 'C00026596', {'weight': 0.703407}) ('C00017792', 'C00028350', {'weight': 0.805869})
dame
('C00017792', 'C00016476', {'weight': 0.74946}) ('C00017792', 'C00015228', {'weight': 0.763485})
dame
('C00017792', 'C00017103', {'weight': 0.655172}) ('C00017792', 'C00016476', {'weight': 0.74946})
dame
('C00017792', 'C0001693

In [43]:
print(l1)

[('C00017671', 'C00017726', {'weight': 0.785714}), ('C00017671', 'C00015228', {'weight': 0.763527}), ('C00026595', 'C00015228', {'weight': 0.973684}), ('C00017726', 'C00015228', {'weight': 0.921053}), ('C00017792', 'C00028350', {'weight': 0.805869}), ('C00017792', 'C00026596', {'weight': 0.703407}), ('C00017792', 'C00016476', {'weight': 0.74946}), ('C00017792', 'C00017103', {'weight': 0.655172}), ('C00017792', 'C00016938', {'weight': 0.938534}), ('C00017792', 'C00015229', {'weight': 0.744422}), ('C00017792', 'C00017544', {'weight': 0.755889}), ('C00017792', 'C00028707', {'weight': 0.828194}), ('C00017792', 'C00017237', {'weight': 0.820175}), ('C00017792', 'C00018127', {'weight': 0.846847}), ('C00017792', 'C00028349', {'weight': 0.917476}), ('C00017792', 'C00015228', {'weight': 0.706827}), ('C00017792', 'C00016751', {'weight': 0.824324}), ('C00017792', 'C00017791', {'weight': 0.720698}), ('C00016476', 'C00017791', {'weight': 0.699029}), ('C00016476', 'C00028349', {'weight': 0.937046}), 

In [80]:
flag = [1] * len(l2)
k = 0
for e1 in l1:
    for i, e2 in enumerate(l2):
        if e1 == e2 and flag[i]:
            # print(e1, "OK")
            flag[i] = 0
            break
        elif e1[0] == e2[1] and e1[1] == e2[0] and e1[2] == e2[2] and flag[i]:
            flag[i] = 0
            break
        elif e1[0] == e2[1] and e1[1] == e2[0] and flag[i]:
            flag[i] = 0
            print(e1[:2])
            print(e1[2], e2[2])
            print("weight違い")
            break
        elif e1[0] == e2[0] and e1[1] == e2[1] and flag[i]:
            flag[i] = 0
            print(e1[:2])
            print(e1[2], e2[2])
            print("weight違い")
            break
    else:
        print(e1)
        k += 1
print(k)
k = 0
for i , e2 in enumerate(l2):
    if flag[i]:
        print(e2)
        k += 1
print(k)

('C00017671', 'C00015228')
{'weight': 0.763527} {'weight': 0.792261}
weight違い
('C00026595', 'C00015228')
{'weight': 0.973684} {'weight': 0.973392}
weight違い
('C00017726', 'C00015228')
{'weight': 0.921053} {'weight': 0.912088}
weight違い
('C00017792', 'C00015228')
{'weight': 0.706827} {'weight': 0.763485}
weight違い
('C00016476', 'C00015228')
{'weight': 0.769547} {'weight': 0.762295}
weight違い
('C00016938', 'C00015228')
{'weight': 0.77551} {'weight': 0.808732}
weight違い
('C00015229', 'C00015228')
{'weight': 0.974359} {'weight': 0.97397}
weight違い
('C00017544', 'C00015228')
{'weight': 0.750503} {'weight': 0.77551}
weight違い
('C00018127', 'C00015228')
{'weight': 0.750503} {'weight': 0.779141}
weight違い
('C00024753', 'C00015228')
{'weight': 0.763527} {'weight': 0.792261}
weight違い
('C00015228', 'C00017791')
{'weight': 0.631579} {'weight': 0.626374}
weight違い
('C00015228', 'C00028349')
{'weight': 0.690141} {'weight': 0.75}
weight違い
0
0


In [48]:
print(len(l1))

102


In [49]:
print(len(G1.edges(data=True)))

190


In [63]:
print(len(G2.edges(data=True)))

190


In [54]:
print(len(l2))

102


数週間前に取った時と値が全然違うやつがある  
SIMCOMP2に入ってるやつが正しそう

In [81]:
f = MCS_Finder("Streptomyces")

In [82]:
all_Clist = f.get_Cnumber(f.get_html(f.gene))

finish getting Cnumber


In [83]:
print(len(all_Clist))

1891


In [84]:
temp = all_Clist[:100]
print(len(temp))

100


In [98]:
%%time
i = 0
t = time.time()
dic = dict()
Cnumber = "C00015228"
filepath = "SIMCOMP2/" + Cnumber + "test.txt"
url = "http://rest.genome.jp/simcomp2/"+ Cnumber + "/"
for tem in temp:
    url += tem + "+"
url = url[:-1]
url += "/cutoff=0.01"
urllib.request.urlretrieve(url, filepath)
i += 1
print(url, i)
print(time.time() - t)
print(time.strftime("%X"))

http://rest.genome.jp/simcomp2/C00015228/C00000108+C00000109+C00000112+C00000114+C00000146+C00000334+C00000336+C00000339+C00000340+C00000368+C00000369+C00000370+C00000371+C00000372+C00000373+C00000374+C00000375+C00000376+C00000377+C00000379+C00000380+C00000381+C00000382+C00000383+C00000384+C00000385+C00000486+C00000730+C00000733+C00000734+C00000740+C00000744+C00000747+C00000748+C00000750+C00000752+C00000753+C00000778+C00000779+C00000780+C00000782+C00000783+C00000785+C00000787+C00000788+C00000789+C00000790+C00000791+C00000793+C00000794+C00000795+C00000796+C00000797+C00000798+C00000799+C00000800+C00000801+C00000846+C00000905+C00000906+C00000907+C00000908+C00000910+C00000911+C00000912+C00000913+C00000914+C00000916+C00000919+C00000926+C00001131+C00001152+C00001203+C00001354+C00001507+C00001775+C00002035+C00002037+C00002059+C00002350+C00002457+C00002461+C00002487+C00002564+C00002662+C00002669+C00002682+C00002845+C00008408+C00008409+C00009380+C00009384+C00009391+C00009475+C00009852+C00011026

この速度なら1900こまとめてやっても平気そう？

In [99]:
%%time
i = 0
t = time.time()
dic = dict()
Cnumber = "C00015228"
filepath = "SIMCOMP2/" + Cnumber + "test.txt"
url = "http://rest.genome.jp/simcomp2/"+ Cnumber + "/"
for tem in all_Clist:
    url += tem + "+"
url = url[:-1]
url += "/cutoff=0.01"
# urllib.request.urlretrieve(url, filepath)
i += 1
print(url, i)
print(time.time() - t)
print(time.strftime("%X"))

http://rest.genome.jp/simcomp2/C00015228/C00000108+C00000109+C00000112+C00000114+C00000146+C00000334+C00000336+C00000339+C00000340+C00000368+C00000369+C00000370+C00000371+C00000372+C00000373+C00000374+C00000375+C00000376+C00000377+C00000379+C00000380+C00000381+C00000382+C00000383+C00000384+C00000385+C00000486+C00000730+C00000733+C00000734+C00000740+C00000744+C00000747+C00000748+C00000750+C00000752+C00000753+C00000778+C00000779+C00000780+C00000782+C00000783+C00000785+C00000787+C00000788+C00000789+C00000790+C00000791+C00000793+C00000794+C00000795+C00000796+C00000797+C00000798+C00000799+C00000800+C00000801+C00000846+C00000905+C00000906+C00000907+C00000908+C00000910+C00000911+C00000912+C00000913+C00000914+C00000916+C00000919+C00000926+C00001131+C00001152+C00001203+C00001354+C00001507+C00001775+C00002035+C00002037+C00002059+C00002350+C00002457+C00002461+C00002487+C00002564+C00002662+C00002669+C00002682+C00002845+C00008408+C00008409+C00009380+C00009384+C00009391+C00009475+C00009852+C00011026

長すぎた。  
適当な長さで切って後でまとめた方が良さそう。

simnet.pyにSIMCOMP2を使うバージョンは入れておくべき。

In [5]:
url = "http://rest.genome.jp/simcomp2/"
urlC = ""
for Cnumber in Clist:
    urlC += Cnumber + "+"
else:
    urlC = urlC[:-1]
url += urlC + "/" + urlC + "/cutoff=0.1"
print(url)

http://rest.genome.jp/simcomp2/C00015228+C00015229+C00016476+C00016751+C00016938+C00017103+C00017237+C00017544+C00017573+C00017671+C00017726+C00017791+C00017792+C00018127+C00024753+C00026595+C00026596+C00028349+C00028350+C00028707/C00015228+C00015229+C00016476+C00016751+C00016938+C00017103+C00017237+C00017544+C00017573+C00017671+C00017726+C00017791+C00017792+C00018127+C00024753+C00026595+C00026596+C00028349+C00028350+C00028707/cutoff=0.1


In [6]:
import urllib

In [8]:
%%time
urllib.request.urlretrieve(url, "SIMCOMP2/testtesttest.txt")

CPU times: user 6.64 ms, sys: 7.1 ms, total: 13.7 ms
Wall time: 33.5 s


('SIMCOMP2/testtesttest.txt', <http.client.HTTPMessage at 0x112f92cf8>)